In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
from time import localtime, strftime

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

In [26]:
# Number of samples in each class

TEST_SAMPLE_NUMBER = 57092
TRAIN_SAMPLE_NUMBER = 285998

# array1: [sX1; sY1; sX2; sY2; ...]
# array2: [eX1; eY1; eX2; eY2; ...]
# output: [dist((sX1, sY1), (eX1, eY1)), dist((sX1, sY1), (eX1, eY1)),
#          dist((sX2, sY2), (eX2, eY2)), dist((sX2, sY2), (eX2, eY2)), ...]

def distance_for_each_point(array1, array2):
    array_length = len(array1)
    len2 = len(array2)
    assert (array_length == len2), "Arrays' sizes have to be equal (array1: {}, array2: {})".format(array_length, len2)
    
    if array1.ndim == 1:
        distance = np.linalg.norm((array1 - array2).reshape((int(array_length / 2), 2)), axis=1)
        result = np.array([[d, d] for d in distance]).flatten()
    else:
        result = np.array([distance_for_each_point(array1[i], array2[i]) for i in range(array_length)])
    
    return result

# metrics between real results (in tests) and predicted
def distance(test_results, predicted_results):
    return distance_for_each_point(np.array(test_results), predicted_results).sum() / TEST_SAMPLE_NUMBER / 2

# data may be present as [n_features * n_samples] or [n_samples * n_features] 
# usually algorithms require second variant but I prefer first
def to_model(df):
    return np.array(df).T

In [9]:
# you need to pass as arguments function to execute, test_results to compare with predicted by function,
# parameter_name to know results by which parameter you want to compare and list_of_values for this parameter
# also you need to pass other argument which function will use 
# it's always some test_data and usually train_data and train_results

def compare_results(function, test_results, parameter_name, list_of_values, **other_parameters):
    result = [] # we will keep results for each configuration here

    for i, value in enumerate(list_of_values):
        print("{cur}/{num}: {t}".format(cur=i+1, num=len(list_of_values), t=strftime("%Y-%m-%d %H:%M:%S", localtime())))
        other_parameters[parameter_name] = value        
        predicted_results = function(**other_parameters)
        np.save("src/Logs/{date}_predicted_coordinates_{function_name}_{parameter_name}_".format(
                date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name)
                + str(value), predicted_results)
        result.append(distance(test_results, predicted_results))
        
    print("done! {time}".format(time=strftime("%Y-%m-%d %H:%M:%S", localtime())))
    print("Results: {}".format(result))
        
    ind = list_of_values
    number = len(ind)
    width = 2 / number
    result_bar = plt.bar(range(number), result, width, color='g')

    plt.ylabel('Average difference')
    plt.xlabel(parameter_name)
    plt.title("Difference between real points and predicted by {parameter_name} in {function_name}".format(
        function_name=function.__name__, parameter_name=parameter_name))
    plt.xticks(np.array(range(number)) + width/2, ind)
    plt.savefig("src/Plots/{date}_{function_name}_difference_by_{parameter_name}.png".format(
        date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name))    

In [220]:
ZERO_PADDING = 6

def get_filename(index):
    return "src/Csv/{}.txt".format(str(index).zfill(ZERO_PADDING))

def get_framename(index):
    return "src/Frame/{}.jpg".format(str(index).zfill(ZERO_PADDING))

NUMBER_OF_PEDESTRIANS = 12273

def download_pedestrian(index):
    assert(0 <= index < NUMBER_OF_PEDESTRIANS), "pedestrian number should be between 0 and {max}; given number: {id}".format(
        max=NUMBER_OF_PEDESTRIANS-1, id=index)
    filename = get_filename(index)
    data = pd.read_csv(filename, index_col=0)
    return data

In [194]:
def find_by_path(path, search_in_test=True):
    
    indeces = np.arange(NUMBER_OF_PEDESTRIANS)
    if search_in_test:
        pedestrian_separation = pd.read_csv('src/pedestrian_labels_and_test_and_train_separation.csv', index_col=0)
        indeces = np.where(np.array(pedestrian_separation['category']) == 'test')[0]
        
    path = path.flatten()    
    len2 = len(path)
        
    for i in indeces:
        df = download_pedestrian(i)
        data = np.array(df).flatten()
        len1 = len(data)
        data_view = np.lib.stride_tricks.as_strided(data, shape=(len1 - len2 + 1, len2),
                                                    strides=(data.dtype.itemsize,) * 2)[::2, :]
        
        ind = np.where(np.all(data_view == path, axis=1))[0]
        if len(ind) > 0:
            return (i, df.index.values[ind[0]])

In [207]:
def make_view(data, length, only_odd_rows=True):
    data = np.array(data).flatten()
    len1 = len(data)
    len2 = length
    return np.lib.stride_tricks.as_strided(data, shape=(len1 - len2 + 1, len2),
                                                    strides=(data.dtype.itemsize,) * 2)[::2 if only_odd_rows else 1, :]

In [3]:
# reading data from .csv files

test_data = pd.read_csv('src/test_data_points.csv', index_col=0)
train_data = pd.read_csv('src/train_data_points.csv', index_col=0)

# split ten frames in input and output data (we want to predict output by input)

test_results = test_data[5:10]
test_data = test_data[:5]

train_results = train_data[5:10]
train_data = train_data[:5]

In [7]:
from sklearn.neighbors import KNeighborsRegressor

# n_neighbors by defalt is 5;         to compare: range(5, 55, 5)
# weights     by defalt is 'uniform'; to compare: ['uniform', 'distance']
# algorithm   by defalt is 'auto';    to compare: ['auto', 'ball_tree', 'kd_tree', 'brute']
# note: 'brute' is causing some memory error

def kneighbors_regressor(train_data, train_results, test_data, **kwargs):
    neigh = KNeighborsRegressor(**kwargs)
    neigh.fit(to_model(train_data), to_model(train_results))
    return (neigh.predict(to_model(test_data))).T

There is no difference which algorithm to use: ball_tree or kd_tree but kd_tree works faster. Also between 'uniform' and 'distance' (parameter weights) there is no difference neither.
The optimal n_neighbors is 25.

Note: I deleted some cells with comparison. You could find them in previous versions of this notebook on github.

In [32]:
predicted = np.load('src/Logs/20161010_predicted_coordinates_kneighbors_regressor_n_neighbors_25.npy')
original = np.array(test_results)
dist = distance_for_each_point(predicted, original)

In [162]:
sum_dist = dist.sum(axis=0) 
indeces = sum_dist.argsort()

In [163]:
sum_dist[indeces[-2:]]

array([ 2489.74389432,  2489.74389432])

In [164]:
predicted[:, indeces[-2:]]

array([[ 757.48,  724.52],
       [ 720.4 ,  784.96],
       [ 687.6 ,  849.  ],
       [ 656.64,  911.92],
       [ 628.64,  969.76]])

In [165]:
original[:, indeces[-2:]]

array([[ 938,  485],
       [ 967,  472],
       [ 998,  455],
       [1015,  429],
       [1039,  417]], dtype=int64)

In [223]:
original_path = original[:, indeces[-2:]]
original_path

array([[ 938,  485],
       [ 967,  472],
       [ 998,  455],
       [1015,  429],
       [1039,  417]], dtype=int64)

In [225]:
predicted_path = predicted[:, indeces[-2:]]
predicted_path

array([[ 757.48,  724.52],
       [ 720.4 ,  784.96],
       [ 687.6 ,  849.  ],
       [ 656.64,  911.92],
       [ 628.64,  969.76]])

In [224]:
pedestrian, first_frame = find_by_path(original_path)

In [226]:
pedestrian

5273

In [227]:
first_frame

59220

In [231]:
def draw_lines(frame_number, paths, colors, image_name):
    im = Image.open(get_framename(frame_number))
    draw = ImageDraw.Draw(im)
    for path, color in zip(paths, colors):
        path_view = make_view(path, 4)
        for line in path_view:
            draw.line(tuple(line), width=4, fill=color)
    im.save("src/Images/{name}.jpg".format(name=image_name))

In [232]:
draw_lines(frame_number=first_frame, paths=[original_path, predicted_path], colors=['green', 'red'], image_name='ped5273')